In [ ]:
# Notes since coping this to Random_t1s folder
# Date: 11.02.25
# -- Part i --
# Note: Can i use the test data (dummy data) to train an RF model to predict income?
# Outcome: Yes. With no data preprocessing, etc. the results look ok.

# -- Part ii --
# Trying with titanic data




# -- Below is from Week 8 --
# Date: 09.02.25
# Note: The Random forest part is awesome and worked off the bat. The interesting part was trying to get the frontier (aka fat) agent going.
# See 2.3 as to why i am using fat (specifically 'fat_price' agent) working. Now, that i have 'fat_price', i can use a single bit of code to 
# get the prices from both solutions :)
# I modified the loop at the end to get 5 comparisons (not 250), but validates the core functionality of the code.


### Part i

In [1]:
import os
import re
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import chromadb
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go

In [2]:
QUESTION = "What is this this applicants income?\n\n"
#DB = "products_vectorstore"
DB = pd.read_csv('test.csv')

In [3]:
modelembed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Remove target col, then embed others
df_targ = DB[['ApplicantIncome']]
df_feat = DB.drop(['ApplicantIncome', 'Loan_ID'], axis=1)


In [ ]:
# Embed each column one by one and then combine (but 1st remove target columns)
print(df_feat.shape)
df_feat["combined_text"] = df_feat.astype(str).agg(" ".join, axis=1)  # Convert all columns to strings and concatenate
df_feat["embeddings"] = df_feat["combined_text"].apply(lambda x: modelembed.encode(x).tolist())  # Convert embeddings to list

df_feat.drop(columns=["combined_text"], inplace=True)  # Remove the intermediate column if not needed
print(df_feat.shape)
display(df_feat.head(2))

(367, 11)
(367, 11)


,Gender,Married,Dependents,Education,Self_Employed,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,embeddings
0,Male,Yes,0,Graduate,No,0,110.0,360.0,1.0,Urban,"[0.058500949293375015, -0.018275007605552673, ..."
1,Male,Yes,1,Graduate,No,1500,126.0,360.0,1.0,Urban,"[0.05155470222234726, -0.017632726579904556, -..."


In [6]:
feat = np.vstack(df_feat['embeddings'].values)
targ = df_targ['ApplicantIncome'].values  # Convert to 1D array

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(feat, targ)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [7]:
df_targ["predicted_income"] = rf_model.predict(feat)
display(df_targ.head(10))

C:\Users\Tim_S\AppData\Local\Temp\ipykernel_15316\898772466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_targ["predicted_income"] = rf_model.predict(feat)


,ApplicantIncome,predicted_income
0,5720,5173.23
1,3076,3283.13
2,5000,4757.71
3,2340,2985.55
4,3276,4990.78
5,2165,2917.08
6,2226,2724.32
7,3881,4242.86
8,13633,10300.90
9,2400,2549.97


In [8]:
# Accuracy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
y_pred = rf_model.predict(feat)  # Predicted values
y_true = targ  # Actual values

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"Mean Absolute Error (MAE): {mae}") # MAE: Average absolute error between predicted and actual values. Lower is better.
print(f"Mean Squared Error (MSE): {mse}") # MSE: Average squared difference between predicted and actual values. Lower is better.
print(f"R² Score: {r2}") # R² Score: Measures how well predictions match actual values (1 is perfect, 0 means no correlation).


Mean Absolute Error (MAE): 962.5828683015441
Mean Squared Error (MSE): 4411343.32700874
R² Score: 0.8165694728886446


In [9]:
# predict a new value
# New input data (adjust as needed)
# Expecting: 5,720
new_data = {
    "Gender": "Male",
    "Married": "Yes",
	"Dependents": 0,
    "Education": "Graduate",
    "Self_Employed": "No",
	"CoapplicantIncome": 0,
    "LoanAmount": 110,
    "Loan_Amount_Term": 360,
	"Credit_History": 1,
    "Property_Area": "Urban"
}
# Convert to text format (same method used in training)
new_text = " ".join(map(str, new_data.values()))
# Generate embedding
new_embedding = modelembed.encode(new_text)
# Reshape to match model input format
new_embedding = np.array(new_embedding).reshape(1, -1)
# Make the prediction
predicted_value = rf_model.predict(new_embedding)
print("Predicted value:", predicted_value[0])


Predicted value: 4957.95


In [15]:
# View vectors
# Let's try 3D!
embeddings = np.array(df_feat['embeddings'].tolist())
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=42, n_jobs=-1)
reduced_vectors = tsne.fit_transform(embeddings)

In [16]:
CATEGORIES = ['Urban', 'Semiurban', 'Rural']
COLORS = ['red', 'blue', 'brown']

categories = df_feat['Property_Area'].tolist()  # Convert Series to list
colors = [COLORS[CATEGORIES.index(c)] for c in categories]  # Assign colors


In [17]:

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=3, color=colors, opacity=0.7),
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [18]:
# 2d version
# 2D version

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=3, color=colors, opacity=0.7),
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

### Part ii

In [26]:
import os
import re
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import chromadb
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import plotly.graph_objects as go

In [42]:
DB = pd.read_csv('train_titanic.csv')
display(DB.head(2))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [43]:
# Missing?
# Count missing values per column
missing_per_column = DB.isna().sum()
print(missing_per_column)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [45]:
# Remove missing (will remove cabin, to keep more rows)
print('pre :', DB.shape)
DBc = DB.drop(['Cabin'], axis=1)
DBc = DBc.dropna()
print('post:', DBc.shape)

pre : (891, 12)
post: (712, 11)


In [46]:
# Remove target col, then embed others
modelembed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

df_targ = DBc[['Survived']]
df_feat = DBc.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1)


In [47]:
# Need to edit below
# Embed each column one by one and then combine (but 1st remove target columns)
print(df_feat.shape)
df_feat["combined_text"] = df_feat.astype(str).agg(" ".join, axis=1)  # Convert all columns to strings and concatenate
df_feat["embeddings"] = df_feat["combined_text"].apply(lambda x: modelembed.encode(x).tolist())  # Convert embeddings to list

df_feat.drop(columns=["combined_text"], inplace=True)  # Remove the intermediate column if not needed
print(df_feat.shape)
display(df_feat.head(2))

(712, 7)
(712, 8)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,embeddings
0,3,male,22.0,1,0,7.2500,S,"[0.007044079713523388, 0.041801292449235916, -..."
1,1,female,38.0,1,0,71.2833,C,"[-0.0073685613460838795, -0.012659283354878426..."


In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(feat, targ, test_size=0.2, random_state=42, stratify=targ)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print results
print(f"Test Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Test Accuracy: 0.7297
Confusion Matrix:
 [[ 6  6]
 [ 4 21]]
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.50      0.55        12
           1       0.78      0.84      0.81        25

    accuracy                           0.73        37
   macro avg       0.69      0.67      0.68        37
weighted avg       0.72      0.73      0.72        37



In [87]:
# predict a new value
# New input data (adjust as needed)
# Expecting: 5,720
new_data = {
    "Pclass": 1,
    "Sex": "male",
	"SibSp": 0,
    "Parch": 1,
    "Fare": 40,
	"Embarked": 0, # C, S Q
}

	
# Convert to text format (same method used in training)
new_text = " ".join(map(str, new_data.values()))
# Generate embedding
new_embedding = modelembed.encode(new_text)
# Reshape to match model input format
new_embedding = np.array(new_embedding).reshape(1, -1)
# Make the prediction
predicted_value = rf_model.predict(new_embedding)
print("Predicted value:", predicted_value[0])

Predicted value: 0


In [57]:
unique_values = df_feat['Embarked'].unique()
print(unique_values)

['S' 'C' 'Q']


In [53]:
# View vectors
# Let's try 3D!
embeddings = np.array(df_feat['embeddings'].tolist())
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=42, n_jobs=-1)
reduced_vectors = tsne.fit_transform(embeddings)

In [85]:
# CATEGORIES = [1, 2, 3, 'male', 'female', 'S', 'C', 'Q']
# COLORS = ['red', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'cyan']

# CATEGORIES = [1, 2, 3, 'male', 'female',]
# COLORS = ['red', 'blue', 'brown', 'orange', 'yellow']

# CATEGORIES = [1, 2, 3]
# COLORS = ['green', 'orange', 'red']

CATEGORIES = ['male', 'female']
COLORS = ['green', 'red']

# CATEGORIES = ['S', 'C', 'Q']
# COLORS = ['red', 'orange', 'green']

categories0 = df_feat['Pclass'].tolist()  # Convert Series to list
categories1 = df_feat['Sex'].tolist()  # Convert Series to list
categories2 = df_feat['Embarked'].tolist()  # Convert Series to list
categories3 = categories0 #+ categories1 #+ categories2
colors = [COLORS[CATEGORIES.index(c)] for c in categories1]  # Assign colors

In [86]:
# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=3, color=colors, opacity=0.7),
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()


In [81]:
# 2d version
# 2D version

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=3, color=colors, opacity=0.7),
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# -----------------------------------------------------------------------
# --- Below is Week 8 day 2 (above is the new code for this notebook) ---
# -----------------------------------------------------------------------

# The Price is Right

Today we build a more complex solution for estimating prices of goods.

1. Day 2.0 notebook: create a RAG database with our 400,000 training data
2. Day 2.1 notebook: visualize in 2D
3. Day 2.2 notebook: visualize in 3D
4. Day 2.3 notebook: build and test a RAG pipeline with GPT-4o-mini
5. Day 2.4 notebook: (a) bring back our Random Forest pricer (b) Create a Ensemble pricer that allows contributions from all the pricers

Phew! That's a lot to get through in one day!

## PLEASE NOTE:

We already have a very powerful product estimator with our proprietary, fine-tuned LLM. Most people would be very satisfied with that! The main reason we're adding these extra steps is to deepen your expertise with RAG and with Agentic workflows.

## Finishing off with Random Forests & Ensemble

In [ ]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
from testing import Tester
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib


In [ ]:
# CONSTANTS
QUESTION = "How much does this cost to the nearest dollar?\n\n"
DB = "products_vectorstore"

QUESTION = "What is this this applicants income?\n\n"
#DB = "products_vectorstore"
DB = pd.read_csv('test.csv')


In [ ]:
# environment
# load_dotenv()
# #os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
# os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Load in the test pickle file:
# with open('test_lite.pkl', 'rb') as file:
#     test = pickle.load(file)

In [ ]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [ ]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
prices = [metadata['price'] for metadata in result['metadatas']]

# Random Forest

We will now train a Random Forest model.

Can you spot the difference from what we did in Week 6? In week 6 we used the word2vec model to form vectors; this time we'll use the vectors we already have in Chroma, from the SentenceTransformer model.

In [ ]:
# This next line takes an hour on my M1 Mac!

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(vectors, prices)

In [ ]:
# Save the model to a file

joblib.dump(rf_model, 'random_forest_model.pkl')

In [ ]:
# Load it back in again

rf_model = joblib.load('random_forest_model.pkl')

In [ ]:
# -- Force reload of agent (need when making change)
import importlib
from agents import fat_price
from agents import fat

importlib.reload(fat)
importlib.reload(fat_price)
from agents.fat import fat
from agents.fat_price import fat_price


In [ ]:
#from agents.specialist_agent import SpecialistAgent
#from agents.frontier_agent import FrontierAgent
from agents.fat import fat
from agents.fat_price import fat_price
from agents.random_forest_agent import RandomForestAgent

In [ ]:
#specialist = SpecialistAgent()
#frontier = FrontierAgent(collection)
fat_instance = fat(collection)
random_forest = RandomForestAgent()

In [ ]:
def description(item):
    return item.prompt.split("to the nearest dollar?\n\n")[1].split("\n\nPrice is $")[0]

In [ ]:
def rf(item):
    return random_forest.price(description(item))

In [ ]:
Tester.test(rf, test)

In [ ]:
product = "Quadcast HyperX condenser mic for high quality audio for podcasting"

In [ ]:
prod2 = test[42]
prod2

In [ ]:
# My code
from agents.fat_price import fat_price
fat_agent = fat_price(collection)
estimated_price = fat_agent.price("Quadcast HyperX condenser mic for high quality audio for podcasting")
print(f"Estimated Price: ${estimated_price:.2f}")

In [ ]:
# This is the code that runs both (i can run the comparison of 250 as i am on the free plan (15 RPM) for flash)
from agents.fat import fat
#item = test[42]
#product = "Quadcast HyperX condenser mic for high quality audio for podcasting"
#product = "Gas Range Oven Bake Igniter Replacement Model Specific Not Universal"
product = "Dryer Heating Element, Replacement with OEM Part Number DC47-00019A and P13312, Dryer Repair Kit Replacement for Old or Broken Heating Element, Easy to Install"

#print(specialist.price(product))
print(fat_agent.price(product))
print(random_forest.price(product))

In [ ]:
# Can't run below for flash

In [ ]:
# specialists = []
# frontiers = []
frontiers = []
random_forests = []
prices = []
#for item in tqdm(test[1000:1250]):
for item in tqdm(test[1000:1005]):    
    text = description(item)
    # specialists.append(specialist.price(text))
    #frontiers.append(frontier.price(text))
    frontiers.append(fat_agent.price(text))
    random_forests.append(random_forest.price(text))
    prices.append(item.price)

In [ ]:
# mins = [min(s,f,r) for s,f,r in zip(specialists, frontiers, random_forests)]
# maxes = [max(s,f,r) for s,f,r in zip(specialists, frontiers, random_forests)]
mins = [min(r) for r in zip(random_forests, frontiers)]
maxes = [max(r) for r in zip(random_forests, frontiers)]

X = pd.DataFrame({
    # 'Specialist': specialists,
    'Frontier': frontiers,
    'RandomForest': random_forests,
    'Min': mins,
    'Max': maxes,
})

# Convert y to a Series
y = pd.Series(prices)

In [ ]:
# Train a Linear Regression
np.random.seed(42)

lr = LinearRegression()
lr.fit(X, y)

feature_columns = X.columns.tolist()

for feature, coef in zip(feature_columns, lr.coef_):
    print(f"{feature}: {coef:.2f}")
print(f"Intercept={lr.intercept_:.2f}")

In [ ]:
joblib.dump(lr, 'ensemble_model.pkl')

In [ ]:
from agents.ensemble_agent import EnsembleAgent
ensemble = EnsembleAgent(collection)

In [ ]:
ensemble.price(product)

In [ ]:
def ensemble_pricer(item):
    return ensemble.price(description(item))

In [ ]:
Tester.test(ensemble_pricer, test)

# WHAT A DAY!

We got so much done - a Fronter RAG pipeline, a Random Forest model using transformer-based encodings, and an Ensemble model.

You can do better, for sure!

Tweak this, and try adding components into the ensemble, to beat my performance.